# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 6h Sleeping Hypothalamus Expression values and Group 2 be 6h SD Hypothalamus Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_HypoT_6h_'   # define a prefix to add to column names (making indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE6514.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 137)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 137)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites',
       ...
       'adjp-HypoT_12hS_vs_HypoT_12hSD', 'GSM149636_HypoT_12hSD.CEL',
       'GSM149637_HypoT_12hSD.CEL', 'GSM149648_HypoT_12hSD.CEL',
       'GSM149649_HypoT_12hSD.CEL', 'GSM149650_HypoT_12hSD.CEL',
       'avg-HypoT_12hSD', 'ANOVA-rawp', 'ANOVA-adjp', 'largest fold'],
      dtype='object', length=137)

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='HypoT_6hS.CEL'
sd_filt ='HypoT_6hSD.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM149611_HypoT_6hS.CEL,GSM149612_HypoT_6hS.CEL,GSM149613_HypoT_6hS.CEL,GSM149614_HypoT_6hS.CEL,GSM149615_HypoT_6hS.CEL
Probesets,,,,,
1427138_at,6.97802,6.71189,6.61429,7.00522,6.58266
1425600_a_at,7.76130,7.64151,7.29653,7.68200,6.51184
1457168_at,5.58342,5.74406,5.74734,5.69504,5.44531
1450135_at,6.04036,5.69884,5.66931,5.84553,5.45209
1424014_at,7.96423,8.07635,8.14194,8.33577,8.07359


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM149616_HypoT_6hSD.CEL,GSM149617_HypoT_6hSD.CEL,GSM149618_HypoT_6hSD.CEL,GSM149619_HypoT_6hSD.CEL,GSM149620_HypoT_6hSD.CEL
Probesets,,,,,
1427138_at,6.79127,6.38225,6.48827,6.78571,6.54389
1425600_a_at,6.84663,6.65625,6.65040,7.51871,6.34114
1457168_at,5.80378,5.91541,5.75264,6.01279,5.36194
1450135_at,5.77172,5.85351,5.59606,5.85735,5.50770
1424014_at,7.56916,7.77570,7.90786,7.72928,8.14480


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at     -0.180138
1425600_a_at   -0.576010
1457168_at      0.126278
1450135_at     -0.023958
1424014_at     -0.293016
Name: 430AV2_HypoT_6h_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at     -0.937968
1425600_a_at   -1.201743
1457168_at      0.636904
1450135_at     -0.125684
1424014_at     -1.620591
Name: 430AV2_HypoT_6h_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at     -0.859804
1425600_a_at   -1.101598
1457168_at      0.583829
1450135_at     -0.115211
1424014_at     -1.485542
Name: 430AV2_HypoT_6h_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.443989
1425600_a_at    0.472209
1457168_at      0.420282
1450135_at      0.400790
1424014_at      0.531316
Name: 430AV2_HypoT_6h_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_HypoT_6h_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_HypoT_6h_Enrich,430AV2_HypoT_6h_poolStDev,430AV2_HypoT_6h_Cohens_d,430AV2_HypoT_6h_J,430AV2_HypoT_6h_Hedges_g,430AV2_HypoT_6h_Var_d,430AV2_HypoT_6h_Var_g,430AV2_HypoT_6h_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1418571_at,Tnfrsf12a,"tumor necrosis factor receptor superfamily, me...",ENSMUSG00000023905,27279,NaN,"substrate-bound cell migration, cell attachmen...",protein binding // receptor activity,cell surface // ruffle // plasma membrane // i...,NaN,"mmu-miR-149(miRanda), mmu-miR-199a-5p(miRanda)...",...,6.663027e-07,0.528016,0.471886,0.074332,6.348331,0.916667,5.819304,2.415065,2.029326,1.424544
1429169_at,Rbm3,RNA binding motif protein 3 [Source:MGI Symbol...,ENSMUSG00000031167,19652,NaN,translation // response to cold // gene silenc...,RNA binding // nucleic acid binding // nucleot...,NaN,GenMAPP-mRNA_processing_binding_Reactome // mR...,"mmu-miR-1(miRanda), mmu-miR-1192(miRanda), mmu...",...,1.205980e-15,1.316408,0.813390,0.135529,6.001574,0.916667,5.501443,2.200945,1.849405,1.359928
1433103_at,Hyls1,hydrolethalus syndrome 1,ENSMUSG00000050555,76832,Mm.252879,NaN,NaN,cytoplasm // nucleus,NaN,"mmu-miR-1192(miRanda), mmu-miR-128(miRanda), m...",...,7.693521e-01,0.258286,0.180958,0.030282,5.975670,0.916667,5.477697,2.185432,1.836370,1.355127
1442614_at,Il1rap,interleukin 1 receptor accessory protein [Sour...,ENSMUSG00000022514,16180,NaN,signal transduction // innate immune response ...,interleukin-1 receptor activity // transmembra...,extracellular region // plasma membrane // int...,IL-1 Signaling Pathway:WP37(WikiPathways),"mmu-miR-1(miRanda), mmu-miR-10a(miRanda), mmu-...",...,4.960067e-20,1.493594,0.488220,0.082159,5.942364,0.916667,5.447167,2.165585,1.819693,1.348960
1418270_at,Adamts8 /// LOC100047263,a disintegrin-like and metallopeptidase (repro...,ENSMUSG00000031994,30806|100047263,NaN,proteolysis,peptidase activity // metalloendopeptidase act...,extracellular region // proteinaceous extracel...,NaN,"mmu-let-7(TargetScan), mmu-let-7a(miRanda), mm...",...,7.504806e-02,0.276828,0.244078,0.041812,5.837500,0.916667,5.351042,2.103820,1.767794,1.329584
1427126_at,Hspa1b,heat shock protein 1B,ENSMUSG00000090877,15511,NaN,response to stress // anti-apoptosis // negati...,protein binding // nucleotide binding // ATP b...,mitochondrion // intracellular // mitochondria...,NaN,"mmu-miR-128(miRanda), mmu-miR-130a(miRanda), m...",...,1.119386e-22,3.350738,2.018582,0.363154,5.558479,0.916667,5.095272,1.944834,1.634201,1.278359
1452388_at,Hspa1a,heat shock protein 1A,ENSMUSG00000091971,193740,NaN,DNA repair // response to stress // response t...,nucleotide binding // ATP binding,mitochondrion // cytoplasmic part,Apoptosis Modulation by HSP70:WP166(WikiPathwa...,"mmu-miR-130a(miRanda), mmu-miR-130b(miRanda), ...",...,6.941324e-16,1.832462,0.886794,0.162709,5.450181,0.916667,4.995999,1.885224,1.584111,1.258615
1435250_at,Ints8,integrator complex subunit 8,ENSMUSG00000040738,72656,NaN,snRNA processing,protein binding,integrator complex // nucleus,NaN,"mmu-let-7a(RNAhybrid|miRanda), mmu-let-7c(RNAh...",...,3.966485e-07,1.078328,0.463760,0.085738,5.409019,0.916667,4.958267,1.862874,1.565332,1.251132
1435342_at,Kcnk6,"potassium inwardly-rectifying channel, subfami...",ENSMUSG00000046410,52150,NaN,ion transport // potassium ion transport // tr...,potassium channel activity // ion channel acti...,integral to membrane // membrane,NaN,"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,4.173710e-02,0.258946,0.227148,0.042370,5.361040,0.916667,4.914287,1.837037,1.543622,1.242426


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites',
       ...
       'ANOVA-adjp', 'largest fold', '430AV2_HypoT_6h_Enrich',
       '430AV2_HypoT_6h_poolStDev', '430AV2_HypoT_6h_Cohens_d',
       '430AV2_HypoT_6h_J', '430AV2_HypoT_6h_Hedges_g',
       '430AV2_HypoT_6h_Var_d', '430AV2_HypoT_6h_Var_g',
       '430AV2_HypoT_6h_SEg'],
      dtype='object', length=145)

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM149516_CerCx_3hS.CEL,GSM149517_CerCx_3hS.CEL,GSM149518_CerCx_3hS.CEL,GSM149519_CerCx_3hS.CEL,GSM149520_CerCx_3hS.CEL,avg-CerCx_3hS,log_fold-CerCx_3hS_vs_CerCx_3hSD,fold-CerCx_3hS_vs_CerCx_3hSD,rawp-CerCx_3hS_vs_CerCx_3hSD,adjp-CerCx_3hS_vs_CerCx_3hSD,...,ANOVA-adjp,largest fold,430AV2_HypoT_6h_Enrich,430AV2_HypoT_6h_poolStDev,430AV2_HypoT_6h_Cohens_d,430AV2_HypoT_6h_J,430AV2_HypoT_6h_Hedges_g,430AV2_HypoT_6h_Var_d,430AV2_HypoT_6h_Var_g,430AV2_HypoT_6h_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_HypoT_6h_Enrich',u'430AV2_HypoT_6h_Hedges_g', u'430AV2_HypoT_6h_Var_g', u'430AV2_HypoT_6h_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_HypoT_6h_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_HypoT_6h_Enrich,430AV2_HypoT_6h_Hedges_g,430AV2_HypoT_6h_Var_g,430AV2_HypoT_6h_SEg
MGI symbol,,,,
0610005C13Rik,0.102502,0.938778,0.380176,0.616584
0610008F07Rik,0.111962,0.605993,0.354473,0.595376
0610009B22Rik,0.251232,0.975051,0.383647,0.619393
0610009D07Rik,0.759258,1.439768,0.439759,0.663143
0610009O20Rik,0.015426,0.118090,0.336808,0.580352
0610010K14Rik,-0.276126,-1.369602,0.429902,0.655669
0610012G03Rik,0.487929,1.401823,0.439447,0.662013
0610031J06Rik,-0.103994,-0.558307,0.351696,0.593040
0610037L13Rik,0.103826,0.880028,0.374834,0.612237
